In [1]:
library(fit)
library(tidyverse)
library(jsonlite)
#gpxsee to inspect fit files

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



# Decoder script
## Helper functions

In [2]:
#input: int from FIT, out: string
get_sport <- function(f){
    case_when(
        f == 2 ~ "Bike",
        f == 17 ~ "Hiking",
        f == 1 ~ "Running",
        f == 5 ~ "Swimming",
        TRUE ~ "UNKNOWN_SPORT"
    )
}

In [3]:
#convert timestamp to date (weird, but off by only some minutes)
to_date <- function (t) {as.POSIXct(as.numeric(as.character(t)), origin="1990-01-01", tz="GMT+23")}

#array: hours k[1], mins k[2], secs k[3] (from int (seconds))
to_duration <- function (t,x="h") {
            if(x!="s"){
                t <- round(t/1000) #normalize if t comes from f.ex total_elapsed_time
            }
            hours <- t%/%3600
            mins <- t%%3600%/%60
            secs <- t%%60
            paste(hours,":",floor(mins),":",floor(secs))
}

#km/h -> min/km
to_pace <- function (t){
    h <- 60/t
    mins <- floor(h)
    secs <- round((h-mins)*60)
    ifelse(secs >= 60, paste(mins+1,":",secs-60),paste(mins,":",secs))
}

#m/s -> km/h
to_kmh <- function (ms) {ms*3.6}

#m/s -> min/100m 
to_swimpace <- function (ms) {
    t <- 10**4/(ms*6) #[sec]
    mins <- floor(t)
    secs <- round((t-mins)*60)
    ifelse(secs >= 60, paste(mins+1,":",secs-60),paste(mins,":",secs))
}

#cadence, time (unit from dfs), poollength [m]
to_swolf <- function (cad,time,length){
    round(cad + (10**5/834)/(10**4/length),1)
}

In [4]:
convert_session <- function(buf,sport){
    if(sport=="Running" | sport=="Hiking"){
        data.frame("avg_hr"            = buf$avg_heart_rate, 
                   "max_hr"            = buf$max_heart_rate,
                   "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
                   "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
                   "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
                   "max_pace"          = to_pace(round(to_kmh(buf$max_speed/1000),1)),
                   "total_ascent"      = buf$total_ascent, # TODO 
                   "total_descent"     = buf$total_descent, # TODO
                   "total_distance"    = round(buf$total_distance/10**5,2),
                   "total_time"        = to_duration(buf$total_elapsed_time),
                   "total_time_moving" = to_duration(buf$total_timer_time),
                   "num_laps"          = buf$num_laps,
                   "training_effect"   = buf$total_training_effect/10
                  )
    }
    else if(sport=="Bike"){
        data.frame("avg_hr"            = buf$avg_heart_rate, 
                   "max_hr"            = buf$max_heart_rate,
                   "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
                   "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
                   "total_ascent"      = buf$total_ascent, # TODO 
                   "total_descent"     = buf$total_descent, # TODO
                   "total_distance"    = round(buf$total_distance/10**5,2),
                   "total_time"        = to_duration(buf$total_elapsed_time),
                   "total_time_moving" = to_duration(buf$total_timer_time),
                   "training_effect"   = buf$total_training_effect/10)
    }
    else if(sport=="Swimming"){
        data.frame("avg_cadence"=buf$avg_cadence, #[Züge/min]
           "total_cycles"=buf$total_cycles, #Gesamtzahl Züge
           "avg_speed"=to_swimpace(buf$avg_speed),
           "max_speed"=to_swimpace(buf$max_speed),
           "avg_swolf"=to_swolf(buf$avg_cadence,buf$avg_speed,buf$pool_length),
           #"avg_stroke_distance"=buf$avg_stroke_distance, #???
           "num_laps"=buf$num_laps,
           "pool_length"=buf$pool_length/100,
           "total_distance"=round(buf$total_distance/10**5,2),#[km]
           "total_time"=to_duration(buf$total_elapsed_time), #[h:m:s]
           "total_time_moving"=to_duration(buf$total_timer_time) #[h:m:s]
          )
    }
}


convert_lap <- function(buf,sport,pool_length=0){
    if(sport=="Running" | sport=="Hiking"){
        data.frame("avg_hr"            = buf$avg_heart_rate,
                   "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
                   "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
                   "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
                   #"time"            = to_date(buf$timestamp),
                   "total_distance"    = round(buf$total_distance/10**5,2),
                   "total_time"        = to_duration(buf$total_elapsed_time),
                   "total_time_moving" = to_duration(buf$total_timer_time))
    }
    else if(sport=="Bike"){
        data.frame("avg_hr"            = buf$avg_heart_rate,
               "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
               "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
               "total_distance"    = round(buf$total_distance/10**5,2),
               "total_time"        = to_duration(buf$total_elapsed_time),
               "total_time_moving" = to_duration(buf$total_timer_time))
    }
    else if(sport=="Swimming"){
        data.frame("avg_cadence"=buf$avg_cadence, #[Züge/min]
               "total_cycles"=buf$total_cycles, #Gesamtzahl Züge
               "avg_speed"=to_swimpace(buf$avg_speed),
               "max_speed"=to_swimpace(buf$max_speed),
               "num_lengths"=buf$num_active_lengths,
               "avg_swolf"=to_swolf(buf$avg_cadence,buf$avg_speed,pool_length),
               "total_distance"=round(buf$total_distance/10**5,2),#[km]
               "total_time"=to_duration(buf$total_elapsed_time), #[h:m:s]
               "total_time_moving"=to_duration(buf$total_timer_time) #[h:m:s]
              ) 
    }
}


convert_record <- function(buf,sport,pool_length=0){
    if(sport=="Running" | sport=="Hiking"){
        data.frame("distance" = round(buf$distance/10**5,3),
               "hr" = buf$heart_rate,
               "speed" = round(to_kmh(buf$speed/1000),1),
               "pace" = to_pace(round(to_kmh(buf$speed/1000),1)),
               "pos_lat" = buf$position_lat,
               "pos_long" = buf$position_long,
               "time" = to_duration(buf$timestamp-file$session$start_time,"s"))
    }
    else if(sport=="Bike"){
        data.frame("distance" = round(buf$distance/10**5,3),
               "hr" = buf$heart_rate,
               "speed" = round(to_kmh(buf$speed/1000),1),
               "pos_lat" = buf$position_lat,
               "pos_long" = buf$position_long,
               "time" = to_duration(buf$timestamp-file$session$start_time,"s"))
    }
    else if(sport=="Swimming"){
        data.frame("cadence"=buf$cadence, #[Züge/min]
           "speed"=to_swimpace(buf$speed),
           "swolf"=to_swolf(buf$cadence,buf$speed,file$session$pool_length),
           "distance"=round(buf$distance/10**5,3),#[km]
           "time"=to_duration(buf$timestamp-file$session$start_time,"s")) #[h:m:s]
    }
}

## Start decoding here!

Plan: Read all files in ACTIVITY directory, and convert all *.FIT files to *.JSON files.

The *.FIT files are deleted then

In [25]:
act_files <- list.files('ACTIVITY/')
#contains all fit files to be processed
fit_files <- c()
for (f in act_files){
    sp <- unlist(strsplit(f,"[.]"))
    #print(sp[1:length(sp)-1])
    if(sp[length(sp)] == "FIT" | sp[length(sp)] == "fit"){
        #file is a fit file! save its name
        fit_files <- c(fit_files,sp[1:length(sp)-1])
    }
}

In [15]:
#test for single files
file <- read.fit(paste('ACTIVITY/',fit_files[59],'.FIT',sep=""))

file_sport_type <- get_sport(file$sport$sport)
file_date <- to_date(file$session$start_time)

file_session <- convert_session(file$session,file_sport_type)
file_lap     <- convert_lap(file$lap,file_sport_type,ifelse(!is.null(file$session$pool_length),
                                                            file$session$pool_length,0))
file_record  <- convert_record(file$record,file_sport_type,ifelse(!is.null(file$session$pool_length),
                                                            file$session$pool_length,0))

ERROR: Error in parse(text = x, srcfile = src): <text>:12:26: Unerwartete(s) ']'
11:                                                             file$session$pool_length,0))
12: fit_files <- fit_files[3:]
                             ^


In [31]:
fit_files[3:20]

[1] "A3JF5121" "A3MH3024" "A3QG0759" "A41F0019" "A45B2025" "A48F1409"
 [7] "A4DH3133" "A4FI2153" "A4GG1541" "A4HE4319" "A4JC1031" "A4KH3745"
[13] "A4NA5618" "A4PF2759" "A4SG1908" "A51G4017" "A53D4500" "A56H1455"

In [33]:
for (f in fit_files[3:4]){
    file <- read.fit(paste('ACTIVITY/',f,'.FIT',sep=""))
    
    file_sport_type <- get_sport(file$sport$sport)
    file_date <- to_date(file$session$start_time)
    file_session <- convert_session(file$session,file_sport_type)
    file_lap     <- convert_lap(file$lap,file_sport_type,ifelse(!is.null(file$session$pool_length),
                                                            file$session$pool_length,0))
    file_record  <- convert_record(file$record,file_sport_type,ifelse(!is.null(file$session$pool_length),
                                                            file$session$pool_length,0))
    out <- c()
    out$session <- file_session
    out$lap <- file_lap
    out$record <- file_record
    
    filename <- paste("ACT_JSONS/",gsub(":","-",gsub(" ","_",file_date)),"_",file_sport_type,".json",sep="")
    write_json(out,filename)
}

In [9]:
out <- c()
out$session <- file_session
out$lap <- file_lap
out$record <- file_record

#save(out, file="test.json")

filename <- paste(gsub(":","-",gsub(" ","_",file_date)),"_",file_sport_type,".json",sep="")

write_json(out,filename)

In [162]:
fromJSON(filename)

$session
  avg_hr max_hr avg_speed max_speed avg_pace max_pace total_ascent
1    125    173       3.8       9.1  15 : 47   6 : 35         1492
  total_descent total_distance  total_time total_time_moving num_laps
1          1496           9.52 2 : 39 : 55       2 : 29 : 51       10
  training_effect
1             3.2

$lap
   avg_hr avg_speed max_speed avg_pace total_distance  total_time
1     116       5.0       5.6   12 : 0           1.00  0 : 12 : 6
2     138       5.5       7.7  10 : 54           1.00 0 : 10 : 54
3     150       3.5       6.6   17 : 8           1.00 0 : 17 : 19
4     125       3.7       9.1  16 : 12           1.00  0 : 16 : 5
5     121       2.8       5.5  21 : 25           1.00 0 : 31 : 38
6     116       4.2       8.8  14 : 17           1.00 0 : 14 : 18
7     116       3.5       5.8   17 : 8           1.00 0 : 16 : 54
8     120       5.0       6.1   12 : 0           1.00  0 : 12 : 3
9     123       3.2       6.0  18 : 45           1.00 0 : 18 : 50
10    127       3.2       5.6  18 : 45           0.52  0 : 9 : 48
   total_time_moving
1         0 : 12 : 6
2        0 : 10 : 54
3        0 : 17 : 19
4         0 : 16 : 5
5        0 : 21 : 45
6        0 : 14 : 18
7        0 : 16 : 54
8         0 : 12 : 3
9        0 : 18 : 39
10        0 : 9 : 48

$record
     distance  hr speed      pace pos_lat pos_long        time
1       0.001 109   4.7   12 : 45 48.5225   9.3510   0 : 0 : 0
2       0.002 109   4.8   12 : 30 48.5225   9.3510   0 : 0 : 1
3       0.010 109   4.3   13 : 57 48.5225   9.3510   0 : 0 : 7
4       0.023 112   4.7   12 : 45 48.5227   9.3510  0 : 0 : 16
5       0.043 113   4.3   13 : 57 48.5228   9.3508  0 : 0 : 34
6       0.064 111   4.1   14 : 38 48.5230   9.3507  0 : 0 : 51
7       0.077 117   4.3   13 : 57 48.5231   9.3507  0 : 0 : 58
8       0.079 120   4.4   13 : 38 48.5231   9.3507  0 : 0 : 59
9       0.081 120   4.5   13 : 20 48.5231   9.3506   0 : 1 : 0
10      0.101 120   4.8   12 : 30 48.5233   9.3505  0 : 1 : 11
11      0.121 117   5.0    12 : 0 48.5235   9.3504  0 : 1 : 23
12      0.135 116   5.2   11 : 32 48.5236   9.3503  0 : 1 : 31
13      0.138 115   5.1   11 : 45 48.5236   9.3503  0 : 1 : 33
14      0.152 120   4.9   12 : 14 48.5235   9.3501  0 : 1 : 43
15      0.155 117   4.9   12 : 14 48.5235   9.3501  0 : 1 : 45
16      0.165 114   4.7   12 : 45 48.5235   9.3499  0 : 1 : 53
17      0.170 111   4.7   12 : 45 48.5235   9.3499  0 : 1 : 57
18      0.171 111   4.7   12 : 45 48.5235   9.3499  0 : 1 : 58
19      0.186 109   4.6    13 : 2 48.5235   9.3497  0 : 2 : 10
20      0.206 112   4.5   13 : 20 48.5235   9.3494  0 : 2 : 26
21      0.228 112   4.7   12 : 45 48.5234   9.3492  0 : 2 : 42
22      0.249 112   4.9   12 : 14 48.5233   9.3489  0 : 2 : 57
23      0.266 115   4.6    13 : 2 48.5232   9.3487  0 : 3 : 13
24      0.277 116   4.4   13 : 38 48.5232   9.3486  0 : 3 : 24
25      0.282 117   4.2   14 : 17 48.5231   9.3485  0 : 3 : 29
26      0.304 115   4.7   12 : 45 48.5231   9.3483  0 : 3 : 41
27      0.324 116   4.9   12 : 14 48.5231   9.3480  0 : 3 : 56
28      0.346 115   5.1   11 : 45 48.5232   9.3477  0 : 4 : 10
29      0.367 112   5.2   11 : 32 48.5233   9.3475  0 : 4 : 23
30      0.388 112   5.3   11 : 19 48.5234   9.3472  0 : 4 : 37
31      0.409 113   5.4    11 : 6 48.5235   9.3470  0 : 4 : 50
32      0.429 116   5.5   10 : 54 48.5237   9.3469   0 : 5 : 3
33      0.442 119   5.5   10 : 54 48.5238   9.3468  0 : 5 : 12
34      0.464 119   5.4    11 : 6 48.5238   9.3465  0 : 5 : 27
35      0.485 120   5.1   11 : 45 48.5237   9.3463  0 : 5 : 45
36      0.506 119   5.0    12 : 0 48.5236   9.3462   0 : 6 : 0
37      0.521 121   4.9   12 : 14 48.5234   9.3461  0 : 6 : 11
38      0.542 121   4.8   12 : 30 48.5234   9.3464  0 : 6 : 28
39      0.563 121   4.7   12 : 45 48.5233   9.3466  0 : 6 : 46
40      0.565 121   4.7   12 : 45 48.5233   9.3467  0 : 6 : 48
41      0.582 118   4.7   12 : 45 48.5232   9.3469   0 : 7 : 0
42      0.596 116   3.7   16 : 12 48.5232   9.3468  0 : 7 : 17


# This part was used to analyze the data decoded by the lib

In [107]:
#rad
data <- read.fit('ACTIVITY/A8FF5035.FIT')
#wandern
data2 <- read.fit('ACTIVITY/A8691324.FIT')
#joggen
data3 <- read.fit('ACTIVITY/A6BG4226.FIT')
#schwimmen
data4 <- read.fit('ACTIVITY/A7CG0124.FIT')

In [108]:
data$session
data2$session
data3$session
data4$session

avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_heart_rate,max_speed,message_index,nec_lat,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
129,4354,4294967295,9,1,0,152,15060,0,48.52735,⋯,621,740,4294967295,588,3161446,8054058,7261254,24,4294967295,0


avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_cadence,max_fractional_cadence,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
56,34,125,650,4294967295,9,1,0,79,64,⋯,907,1481,8670,879,1144476,28127288,17603601,26,4294967295,0


avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_cadence,max_fractional_cadence,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
80,78,160,2589,4294967295,9,1,0,86,0,⋯,113,357,2370,124,460498,1778749,1778749,38,4294967295,0


avg_cadence,avg_speed,avg_stroke_distance,event,event_type,first_lap_index,max_speed,message_index,num_active_lengths,num_laps,⋯,sport,start_time,sub_sport,timestamp,total_calories,total_cycles,total_distance,total_elapsed_time,total_timer_time,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
22,852,237,9,1,0,4124,0,36,12,⋯,5,963496884,17,963499487,383,760,180000,2602139,2596952,0


In [109]:
10**4/(852*6)

[1] 1.956182

In [110]:
head(data$lap)
head(data2$lap)
head(data3$lap)
data4$lap

,avg_heart_rate,avg_speed,end_position_lat,end_position_long,event,event_type,lap_trigger,max_heart_rate,max_speed,message_index,⋯,start_position_long,start_time,sub_sport,timestamp,total_ascent,total_calories,total_descent,total_distance,total_elapsed_time,total_timer_time
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,129,4354,48.48473,9.208667,9,1,7,152,15060,0,⋯,9.199338,966433835,0,966441914,621,740,588,3161446,8054058,7261254


,avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,end_position_lat,end_position_long,event,event_type,lap_trigger,max_cadence,⋯,sub_sport,timestamp,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_work
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,60,28,118,582,47.03492,11.69366,9,1,2,78,⋯,0,965634389,208,179,906,6,1e+05,1981952,1719514,NA
2,62,103,145,543,47.03854,11.68657,9,1,2,79,⋯,0,965637295,217,278,1025,21,1e+05,2906163,1841374,NA
3,62,22,140,644,47.04448,11.69026,9,1,2,79,⋯,0,965641927,150,174,795,7,1e+05,4631508,1553388,NA
4,59,113,134,738,47.05234,11.69249,9,1,2,79,⋯,0,965643553,69,113,726,29,1e+05,1626428,1354143,NA
5,60,122,141,653,47.05937,11.69552,9,1,2,79,⋯,0,965645193,99,165,780,21,1e+05,1639549,1532290,NA
6,59,79,134,495,47.06654,11.69473,9,1,2,79,⋯,0,965647354,84,160,761,17,1e+05,2161168,2019534,NA


,avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,end_position_lat,end_position_long,event,event_type,lap_trigger,max_cadence,⋯,sub_sport,timestamp,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_work
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,82,97,132,2737,48.52356,9.191906,9,1,2,86,⋯,0,960821314,14,58,501,21,100000,365391,365391,NA
2,82,69,150,2629,48.52289,9.183397,9,1,2,84,⋯,0,960821695,31,71,523,7,100000,380373,380373,4294967295
3,81,114,177,2793,48.52788,9.189603,9,1,2,84,⋯,0,960822054,11,81,489,35,100000,358050,358050,4294967295
4,78,55,171,2375,48.53039,9.195290,9,1,2,84,⋯,0,960822475,45,91,538,41,100000,421078,421078,4294967295
5,76,61,174,2383,48.52739,9.199340,9,1,7,85,⋯,0,960822746,12,56,318,20,60498,253857,253857,NA


avg_cadence,avg_speed,avg_stroke_distance,end_position_lat,end_position_long,event,event_type,first_length_index,lap_trigger,max_speed,⋯,start_time,sub_sport,swim_stroke,timestamp,total_calories,total_cycles,total_distance,total_elapsed_time,total_timer_time,total_work
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21,929,263,180,180,9,1,0,0,929,⋯,963496884,17,0,963496938,10,19,5000,53822,53822,4294967295
0,0,0,180,180,9,1,1,0,0,⋯,963496938,17,NA,963496962,0,0,0,23160,23160,4294967295
23,802,211,180,180,9,1,2,0,1004,⋯,963496961,17,5,963497523,99,213,45000,561212,561212,4294967295
0,0,0,180,180,9,1,11,0,0,⋯,963497522,17,NA,963497623,0,0,0,99629,99629,4294967295
21,829,239,180,180,9,1,12,0,3846,⋯,963497622,17,5,963498406,135,272,65000,783879,783879,4294967295
0,0,0,180,180,9,1,25,0,0,⋯,963498406,17,NA,963498535,0,0,0,128324,128324,4294967295
22,992,276,180,180,9,1,26,0,4124,⋯,963498534,17,5,963498988,99,163,45000,453678,453678,4294967295
0,0,0,180,180,9,1,35,0,0,⋯,963498988,17,NA,963499067,0,0,0,78164,78164,4294967295
19,693,217,180,180,9,1,36,0,779,⋯,963499066,17,0,963499211,20,46,10000,144211,144211,4294967295


In [111]:
head(data$record)
head(data2$record)
head(data3$record)
head(data4$record)

,accumulated_power,altitude,distance,heart_rate,position_lat,position_long,speed,timestamp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,65535,4308,0,92,48.52730,9.199338,0,966433835
2,65535,4308,26,92,48.52730,9.199340,0,966433836
3,65535,4308,41,93,48.52730,9.199341,0,966433838
4,65535,4310,41,92,48.52731,9.199309,0,966433850
5,65535,4310,549,92,48.52734,9.199248,1269,966433855
6,65535,4311,962,92,48.52735,9.199193,1997,966433857


,accumulated_power,altitude,cadence,distance,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,65535,NA,0,0,0,101,NA,NA,0,965632404
2,65535,NA,0,0,0,100,NA,NA,0,965632405
3,65535,11398,50,1449,64,103,47.03054,11.69619,1241,965632422
4,65535,11412,53,2180,64,106,47.03058,11.69630,1446,965632428
5,65535,11418,54,3437,64,109,47.03067,11.69641,1334,965632438
6,65535,11424,51,5582,0,107,47.03083,11.69657,1372,965632452


,accumulated_power,altitude,cadence,distance,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,65535,NA,0,142,0,86,NA,NA,1418,960820946
2,65535,NA,0,142,0,86,NA,NA,0,960820947
3,65535,NA,85,142,0,89,NA,NA,0,960820955
4,65535,NA,85,2224,0,91,NA,NA,3518,960820961
5,65535,NA,85,3966,64,94,NA,NA,3415,960820966
6,65535,NA,85,4306,64,98,NA,NA,3396,960820967


,cadence,distance,speed,timestamp
,<dbl>,<dbl>,<dbl>,<dbl>
1,21,5000,929,963496938
2,NA,5000,NA,963496962
3,22,50000,703,963497523
4,NA,50000,NA,963497623
5,21,105000,814,963498304
6,21,115000,843,963498406


In [112]:
#attribute units
attr(data4$session, 'units')

[1] "rpm"     "m/s"     "m"       ""        ""        ""        "m/s"    
 [8] ""        "lengths" ""        "m"       ""        ""        ""       
[15] ""        "s"       "kcal"    "cycles"  "m"       "s"       "s"      
[22] ""

In [113]:
to_kmh(4354/1000)
to_kmh(15060/1122)

to_duration(data4$session$total_elapsed_time)
data4$session$total_distance / 10**5

to_date(data4$session$timestamp)

[1] 15.6744

[1] 48.32086

[1] "0 : 43 : 22"

[1] 1.8

[1] "2020-07-12 15:44:47 GMT"